In [ ]:
import math

def compute_metaldecking(data, fc=20.68, fy=276, cover=20):
    '''
    fc = Compressive Concrete Strength 20.68 MPa
    fy = Steel Bar Yield Strength 276 MPa
    '''
    mm_to_m = 0.001
    UW_CONC = 23.56
    UW_STEEL = 77
    WIDTH = 1000  # TODO - tributary width to compute uniform load
    BEAM_WIDTH = 350  # TODO - to compute clear span
    EFFECTIVE_WIDTH = 0.995 # https://atepsmr.tripod.com/dns-deck4.html

    # --------------------------------------------------------------------------
    # Total Nominal Width of Steel Deck DN-4 in mm
    # --------------------------------------------------------------------------
    if data[5] == 0.8:
        nominal_width = 1021
    elif data[5]  == 1.0:
        nominal_width = 1276
    elif data[5] == 1.2:
        nominal_width = 1532
    elif data[5] == 1.6:
        nominal_width = 2042
    else:
        raise ValueError("Invalid steel deck thickness")

    # --------------------------------------------------------------------------
    # "Not Applicable" or "Input Beam Width"
    # --------------------------------------------------------------------------
    if data[2] == 1:
        widthbeam = 0  # not applicable
    else:
        widthbeam = BEAM_WIDTH

    # --------------------------------------------------------------------------
    # Compute slab permanent clear span in mm
    # --------------------------------------------------------------------------
    if data[9] == 0:
        if data[2] == 1:
            span = data[1]
        else:
            span = data[1] - widthbeam
    else:
        if data[2] == 1:
            span = data[1] / data[9]
        else:
            span = (data[1] / data[9]) - widthbeam

    # --------------------------------------------------------------------------
    # Compute uniform live load per meter strip of slab in kN/m
    # wll = (LL in kN/m^2) × (f) × (panel width in mm) × (0.001)
    # --------------------------------------------------------------------------
    wll = data[7] * data[11] * WIDTH * mm_to_m

    # --------------------------------------------------------------------------
    # Compute distance of top bar from bottom of slab
    # (slab thickness - cover - topbar/2)
    # --------------------------------------------------------------------------
    dprime = data[4] - cover - data[15] / 2

    # --------------------------------------------------------------------------
    # Section calculated data
    # --------------------------------------------------------------------------
    asd = nominal_width * data[5] / EFFECTIVE_WIDTH 
    abb = (data[13] / EFFECTIVE_WIDTH) * math.pi * math.pow(data[12], 2) / 4
    aconc = data[4] * 1000 - 18480 #TODO 18480
    es = 200000
    ec = 4700 * math.sqrt(fc)
    nmod = es / ec

    # moment of inertia of steel deck per meter
    if data[5] == 0.8:
        isd = 304581.947
    elif data[5] == 1.0:
        isd = 373367.871
    elif data[5] == 1.2:
        isd = 442153.796
    elif data[5] == 1.6:
        isd = 579725.646
    else:
        raise ValueError("Invalid steel deck thickness")

    sn = isd / (50 - 18.333)
    sp = isd / 18.333
    wsd = UW_STEEL * asd / 1000000
    wconc = aconc * UW_CONC / 1000000
    wsidl = data[6] + (data[3] / 1000) * 23.56
    wcl = 1.4715
    w1 = 1.6 * wconc + 1.2 * wsd + 1.4 * wcl
    wp = 1.6 * wll + 1.2 * (wsidl + wconc + wsd)

    # --------------------------------------------------------------------------
    # Temporary props
    # --------------------------------------------------------------------------
    spanprop = span / (data[10] + 1)
    fip1 = 0.125 * w1 * math.pow(spanprop, 2) / 0.95 / sn
    fip2 = 0.08 * w1 * math.pow(spanprop, 2) / 0.95 / sn

    if data[10] == 0:
        fipa = fip1
    else:
        fipa = fip2

    if fipa < 0.95 * data[8]:
        fip = fipa
    else:
        fip = data[8]

    # --------------------------------------------------------------------------
    # One meter strip cracked slab section for positive bending
    # --------------------------------------------------------------------------
    ax = WIDTH / (2 * nmod)
    bx = asd + abb
    cx = asd * (data[4] - 22.149) + abb * (data[4] - data[14])
    ac = (-bx + math.sqrt(bx**2 + 4 * ax * cx)) / (2 * ax)

    ic = (
        (1000 / 3 / nmod) * ac**3
        + isd
        + asd * (data[4] - ac - 22.149) ** 2
        + abb * (data[4] - ac - data[14]) ** 2
    )
    sc = ic / (data[4] - ac)

    # --------------------------------------------------------------------------
    # One meter strip uncracked slab section for positive bending
    # --------------------------------------------------------------------------
    b2 = 534
    aux = (WIDTH - b2) / (2 * nmod)
    bux = (asd + abb) + (b2 * data[4] / nmod)
    cux = (
        asd * (data[4] - 22.149)
        + abb * (data[4] - data[14])
        + b2 * (data[4] ** 2) / (2 * nmod)
    )
    auc = (-bux + math.sqrt(bux**2 + 4 * aux * cux)) / (2 * aux)

    iuc = (
        (1000 / 3 / nmod) * auc**3
        + asd * (data[4] - auc - 22.149) ** 2
        + abb * (data[4] - auc - data[14]) ** 2
        + (534 / 3 / nmod) * (data[4] - auc) ** 3
    )
    suc = iuc / 22.149

    # --------------------------------------------------------------------------
    # Average Cracked and Uncracked Moment of Inertia
    # --------------------------------------------------------------------------
    iave = (iuc + ic) / 2

    # --------------------------------------------------------------------------
    # Moments
    # --------------------------------------------------------------------------
    mp_support = data[16] * wp * math.pow((span / 1000),2) #TODO
    if data[9] == 0:
        mp_midspan = 0.125 * wp * math.pow((span / 1000),2) #TODO
    else:
        mp_midspan = 0.08 * wp * math.pow((span / 1000),2) #TODO

    # --------------------------------------------------------------------------
    # Concrete Stress
    # --------------------------------------------------------------------------
    fc_calc = mp_midspan * ac / ic / nmod * 1_000_000 # TODO review

    allowablefc = 0.85 * fc_calc
    concretestress = fc_calc / allowablefc
    if concretestress < 1:
        note_conc = "SAFE"
    else:
        note_conc = "NOT SAFE"

    # --------------------------------------------------------------------------
    # Steel Deck Stress
    # --------------------------------------------------------------------------
    fsd = mp_midspan * 1_000_000 * (data[4] - ac) / ic
    allowablefsd = (0.85 * data[8] - fip)
    if allowablefsd < 0:
        steeldeckstress = fsd / allowablefsd * (-1)
    else:
        steeldeckstress = fsd / allowablefsd
    if steeldeckstress < 1:
        note_steeldeck = "SAFE"
    else:
        note_steeldeck = "NOT SAFE"

    # --------------------------------------------------------------------------
    # Steel Bar Stress
    # --------------------------------------------------------------------------
    fs = mp_midspan * 1_000_000 * (data[4] - ac - data[14]) / ic
    allowablefs = 0.9 * fy
    steelbarstress = fs / allowablefs
    if steelbarstress < 1:
        note_steelbar = "SAFE"
    else:
        note_steelbar = "NOT SAFE"

    # --------------------------------------------------------------------------
    # Check Deflection in mm
    # --------------------------------------------------------------------------
    deflection_LL = 0.013 * wll * (span ** 4) / (es * iave)
    allowable_deflection_LL = span / 360
    ratio_LL = deflection_LL / allowable_deflection_LL
    if ratio_LL < 1:
        note_deflection_LL = "SAFE"
    else:
        note_deflection_LL = "NOT SAFE"

    deflection = 0.013 * (wll + wsd + wconc + wsidl) * (span ** 4) / (es * iave)
    allowable_deflection = span / 240
    ratio = deflection / allowable_deflection
    if ratio < 1:
        note_deflection = "SAFE"
    else:
        note_deflection = "NOT SAFE"

    # --------------------------------------------------------------------------
    # Design of Negative Reinforcing
    # --------------------------------------------------------------------------
    ax_neg = fy / (2 * 0.85 * fc * 500)
    bx_neg = -dprime
    cx_neg = mp_support * 1_000_000 / 0.9 / fy
    as_neg = (-bx_neg - math.sqrt(bx_neg**2 - 4 * ax_neg * cx_neg)) / (2 * ax_neg)
    if data[9] == 0:
        nbar_neg = 0
    else:
        nbar_neg = as_neg / (math.pi * (data[15] ** 2) / 4)

    if nbar_neg == 0:
        sneg = 0
    else:
        sneg = 1000 / nbar_neg

    aww = 0.002 * aconc
    nww = aww / (math.pi * (data[17] ** 2) / 4)
    sww = 1000 / nww

    results = {
        "widthbeam": widthbeam,
        "span": span,
        "wll": wll,
        "dprime": dprime,
        "isd": isd,
        "sn": sn,
        "sp": sp,
        "wsd": wsd,
        "wconc": wconc,
        "wsidl": wsidl,
        "wcl": wcl,
        "w1": w1,
        "wp": wp,
        "spanprop": spanprop,
        "fip1": fip1,
        "fip2": fip2,
        "fip": fip,
        "ax": ax,
        "bx": bx,
        "cx": cx,
        "ac": ac,
        "ic": ic,
        "sc": sc,
        "aux": aux,
        "bux": bux,
        "cux": cux,
        "auc": auc,
        "iuc": iuc,
        "suc": suc,
        "iave": iave,
        "mp_support": mp_support,
        "mp_midspan": mp_midspan,
        "fc": fc_calc,
        "allowablefc": allowablefc,
        "concretestress": concretestress,
        "note_concretestress": note_conc,
        "fsd": fsd,
        "allowablefsd": allowablefsd,
        "steeldeckstress": steeldeckstress,
        "note_steeldeck": note_steeldeck,
        "fs": fs,
        "allowablefs": allowablefs,
        "steelbarstress": steelbarstress,
        "note_steelbar": note_steelbar,
        "deflection_LL": deflection_LL,
        "allowable_deflection_LL": allowable_deflection_LL,
        "ratio_LL": ratio_LL,
        "note_deflection_LL": note_deflection_LL,
        "deflection": deflection,
        "allowable_deflection": allowable_deflection,
        "ratio": ratio,
        "note_deflection": note_deflection,
        "ax_neg": ax_neg,
        "bx_neg": bx_neg,
        "cx_neg": cx_neg,
        "as_neg": as_neg,
        "sneg": sneg,
        "aww": aww,
        "nww": nww,
        "sww": sww,
    }
    return results


if __name__ == "__main__":
    fysd = 550
    beam_type = 1 # 1 = concrete beam ; 2 = steel beam
    slab_data = [
        # [ "name[0]",
        #   "length[1]",
        #   beam_type[2],
        #   "topping[3]",
        #   "slab_thk[4]",
        #   "steel_deck_thk[5]", 0.6, 0.8, 1.0, 1.2, 1.6
        #   "sdl[6]",
        #   "liveload[7]",
        #   fysd[8],
        #   nspan[9],
        #   nprop[10],
        #   f_vibration[11],
        #   botbar[12],
        #   nbot[13],
        #   d_botbar[14],
        #   topbar[15],
        #   coef[16],
        #   dww[17]
        # ]
        ['A', 1.725, beam_type, 50, 125, 1, 1.52, 4.8,  fysd, 1, 2, 1, 0, 0, 25, 12, 0.1, 10],
        ['B', 1.725, beam_type, 50, 125, 1, 1.52, 4.8,  fysd, 1, 2, 1, 0, 0, 25, 12, 0.1, 10],
        ['B', 1.725, beam_type, 50, 125, 1, 1.52, 4.8,  fysd, 1, 2, 1, 0, 0, 25, 12, 0.1, 10],
        ['C', 2.356, beam_type, 50, 125, 1, 1.52, 4.8,  fysd, 1, 2, 1, 0, 0, 25, 12, 0.1, 10],
        ['D', 2.356, beam_type, 50, 125, 1, 1.52, 4.8,  fysd, 1, 2, 1, 0, 0, 25, 12, 0.1, 10],
        ['E', 1.500, beam_type, 50, 125, 1, 1.52, 4.8,  fysd, 1, 2, 1, 0, 0, 25, 12, 0.1, 10],
        ['F', 1.725, beam_type, 50, 125, 1, 1.52, 4.8,  fysd, 1, 2, 1, 0, 0, 25, 12, 0.1, 10],
        ['G', 1.725, beam_type, 50, 125, 1, 1.52, 4.8,  fysd, 1, 2, 1, 0, 0, 25, 12, 0.1, 10],
        ['H', 2.356, beam_type, 50, 125, 1, 1.52, 4.8,  fysd, 1, 2, 1, 0, 0, 25, 12, 0.1, 10],
        ['I', 2.356, beam_type, 50, 125, 1, 1.52, 4.8,  fysd, 1, 2, 1, 0, 0, 25, 12, 0.1, 10],
        ['J', 1.500, beam_type, 50, 125, 1, 1.52, 4.8,  fysd, 1, 2, 1, 0, 0, 25, 12, 0.1, 10],
    ]
    i = 0
    sd_length = len(slab_data)
    while i < sd_length:
        name = slab_data[i][0]
        sd_results = compute_metaldecking(slab_data[i])["note_steeldeck"]
        conc_results = compute_metaldecking(slab_data[i])["note_concretestress"]
        deflection_results = compute_metaldecking(slab_data[i])["note_deflection"]
        print(name + " ; SD = " + str(sd_results) + "; CONC = " + str(conc_results) + "; DEF = " + str(deflection_results))

        i += 1


A ; SD = SAFE; CONC = NOT SAFE; DEF = SAFE
B ; SD = SAFE; CONC = NOT SAFE; DEF = SAFE
B ; SD = SAFE; CONC = NOT SAFE; DEF = SAFE
C ; SD = SAFE; CONC = NOT SAFE; DEF = SAFE
D ; SD = SAFE; CONC = NOT SAFE; DEF = SAFE
E ; SD = SAFE; CONC = NOT SAFE; DEF = SAFE
F ; SD = SAFE; CONC = NOT SAFE; DEF = SAFE
G ; SD = SAFE; CONC = NOT SAFE; DEF = SAFE
H ; SD = SAFE; CONC = NOT SAFE; DEF = SAFE
I ; SD = SAFE; CONC = NOT SAFE; DEF = SAFE
J ; SD = SAFE; CONC = NOT SAFE; DEF = SAFE
